# Cuaderno de Ingesta de Datos

En este bloque traeremos la información de datos abiertos. 

In [0]:
# Paso 1: Descargar los datos con requests y leerlos en pandas
import requests
import pandas as pd
from io import StringIO
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [0]:
spark.sql("USE CATALOG main")

In [0]:
spark.catalog.listDatabases()

In [0]:
# Descargar MEN
url_men = "https://www.datos.gov.co/resource/nudc-7mev.csv?$limit=100000"

response_men = requests.get(url_men)
df_men_pd = pd.read_csv(StringIO(response_men.text))
df_men = spark.createDataFrame(df_men_pd)

display(df_men)

In [0]:
spark.sql("CREATE SCHEMA IF NOT EXISTS main.diplomado_datos")

df_men.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("main.diplomado_datos.men_estadisticas")


In [0]:
total_registros = 19449349
limite = 50000     
paginas = (total_registros // limite) + 1


for i in range(paginas):
    offset = i * limite
    url_secop = f"https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit={limite}&$offset={offset}"

    print(f"Descargando página {i+1}/{paginas} (offset={offset})...")

    response_secop = requests.get(url_secop)

    if response_secop.status_code == 200:
        df_secop_pd = pd.read_csv(StringIO(response_secop.text), dtype=str)

        if df_secop_pd.empty:
            print(f"⚠️ Página {i+1} vacía. Fin de datos.")
            break

        df_secop_spark = spark.createDataFrame(df_secop_pd)

        df_secop_spark.write.format("delta") \
            .mode("append") \
            .option("mergeSchema", "true") \
            .saveAsTable("main.diplomado_datos.secop")

        print(f"✓ Página {i+1} guardada con {len(df_secop_pd)} filas.")
    else:
        print(f"⚠️ Error HTTP {response_secop.status_code}. Deteniendo proceso.")
        break

print("✅ ¡Carga completa de SECOP en Delta!")

In [0]:
spark = SparkSession.builder.getOrCreate()

# Offset donde falló
offset_inicial = 6500000      # Página 131 (6500000 = (131-1)*50000)
limite = 50000

# Calcula cuántas páginas faltan:
total_registros = 19449349
paginas_faltantes = (total_registros - offset_inicial) // limite + 1

print(f"➡️ Vamos a descargar {paginas_faltantes} páginas (bloques de {limite}) empezando en offset {offset_inicial}...")

In [0]:
# --------------------------------------------
# Loop para continuar descarga
# --------------------------------------------

for i in range(paginas_faltantes):
    offset = offset_inicial + (i * limite)
    url_secop = f"https://www.datos.gov.co/resource/rpmr-utcd.csv?$limit={limite}&$offset={offset}"
    
    print(f"Descargando página {i+1} con offset={offset}...")

    response_secop = requests.get(url_secop)

    if response_secop.status_code == 200:
        df_secop_pd = pd.read_csv(StringIO(response_secop.text), dtype=str)

        if df_secop_pd.empty:
            print(f"⚠️ Página {i+1} vacía. Fin de datos.")
            break

        df_secop_spark = spark.createDataFrame(df_secop_pd)

        # Graba en Delta en modo append
        df_secop_spark.write.format("delta") \
            .mode("append") \
            .option("mergeSchema", "true") \
            .saveAsTable("main.diplomado_datos.secop")

        print(f"✓ Página {i+1} guardada con {len(df_secop_pd)} filas.")
    else:
        print(f"⚠️ Error HTTP {response_secop.status_code} en página {i+1}. Deteniendo proceso.")
        break

print("✅ ¡Carga reanudada de SECOP completada!")
